# ResNet Training

In [1]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Move to your current working directory
%cd drive/MyDrive/image_processing

Mounted at /content/drive
/content/drive/MyDrive/image_processing


In [2]:
import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device', device)

Device cuda


In [3]:
import os
import pandas as pd
import numpy as np
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as T
import torch.optim as optim
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset, DataLoader

## Load Images

In [4]:
class CustomDataset(Dataset):
  def __init__(self, image_dir, transform):
    super().__init__()
    self.image_dir = image_dir
    self.transform = transform
    self.images = os.listdir(self.image_dir)

  def __len__(self):
    return len(self.images)

  def __getitem__(self, i):
    image_path = os.path.join(self.image_dir, self.images[i])
    image = Image.open(image_path).convert('L')
    image = self.transform(image)
    label = int(self.images[i][1])
    return (image, label)

In [5]:
training_data = CustomDataset('img_span12/training', T.Compose([T.Resize((224, 224)), T.ToTensor()]))
testing_data = CustomDataset('img_span12/testing', T.Compose([T.Resize((224, 224)), T.ToTensor()]))

In [6]:
print('training images:', len(training_data))
print('training images:', len(testing_data))

training images: 7746
training images: 2350


In [7]:
print(training_data[0])
print(testing_data[0])

(tensor([[[0.0000, 0.7059, 0.4902,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.7059, 0.4902,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.7059, 0.4902,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.7059, 0.4902,  ..., 0.4902, 0.7059, 0.0000],
         [0.0000, 0.7059, 0.4902,  ..., 0.4902, 0.7059, 0.0000],
         [0.0000, 0.7059, 0.4902,  ..., 0.4902, 0.7059, 0.0000]]]), 0)
(tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.7059, 0.4902,  ..., 0.2118, 0.3020, 0.0000],
         [0.0000, 0.7059, 0.4902,  ..., 0.4196, 0.6039, 0.0000],
         [0.0000, 0.7059, 0.4902,  ..., 0.4902, 0.7059, 0.0000]]]), 0)


## Build ResNet Model

In [8]:
class ResNetModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.resnet = models.resnet18(pretrained=True)
    self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    # Remove the final fully connected layer of ResNet
    self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])
    # Define the new fully connected layers
    self.dropout = nn.Dropout(0.5)
    self.fc1 = nn.Linear(512, 10)
    self.fc2 = nn.Linear(512, 2)

  def forward(self, x):
    x = self.resnet(x)
    x = x.view(x.size(0), -1) # Flatten
    x = self.dropout(x)
    out1 = self.fc1(x)
    out2 = self.fc2(x)
    return out1, out2

In [9]:
model = ResNetModel()
print(model)

model = model.cuda()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 191MB/s]


ResNetModel(
  (resnet): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

## Training

In [10]:
# Constants for model training process
BATCH_SIZE = 128
NUM_EPOCHS = 10
PRINT_EVERY = 10

In [11]:
train_loader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)

In [12]:
next(iter(train_loader))

[tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.7059, 0.4902,  ..., 0.4902, 0.7059, 0.0000],
           [0.0000, 0.7059, 0.4902,  ..., 0.4902, 0.7059, 0.0000],
           [0.0000, 0.7059, 0.4902,  ..., 0.4902, 0.7059, 0.0000]]],
 
 
         [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.7059, 0.4902,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.7059, 0.4902,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.7059, 0.4902,  ..., 0.0000, 0.0000, 0.0000]]],
 
 
         [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000

In [13]:
# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [14]:
for epoch in range(NUM_EPOCHS):
  training_acc = 0
  total = 0
  model.train()
  for counter, (x, y) in enumerate(train_loader):
    x, y = x.to(device), y.to(device)
    _, scores = model(x)
    loss = loss_function(scores, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    training_acc += scores.max(1)[1].eq(y).sum().item()
    total += y.size(0)
    if counter % PRINT_EVERY == 0:
      print(f'Epoch[{epoch+1}], Batch[{counter}], Loss: {loss.item()}, Training acc: {training_acc/total}')
  print(f'Epoch[{epoch+1}] completed. Training acc: {training_acc/total}')


Epoch[1], Batch[0], Loss: 1.00953209400177, Training acc: 0.46875
Epoch[1], Batch[10], Loss: 0.8830037117004395, Training acc: 0.5056818181818182
Epoch[1], Batch[20], Loss: 0.767914354801178, Training acc: 0.5152529761904762
Epoch[1], Batch[30], Loss: 0.7390167117118835, Training acc: 0.5073084677419355
Epoch[1], Batch[40], Loss: 0.7451528310775757, Training acc: 0.5099085365853658
Epoch[1], Batch[50], Loss: 0.7109372615814209, Training acc: 0.5107230392156863
Epoch[1], Batch[60], Loss: 0.7092465162277222, Training acc: 0.5143299767621998
Epoch[1] completed. Training acc: 0.5143299767621998
Epoch[2], Batch[0], Loss: 0.720018744468689, Training acc: 0.515625
Epoch[2], Batch[10], Loss: 0.673079252243042, Training acc: 0.5440340909090909
Epoch[2], Batch[20], Loss: 0.72197026014328, Training acc: 0.5368303571428571
Epoch[2], Batch[30], Loss: 0.6802460551261902, Training acc: 0.5451108870967742
Epoch[2], Batch[40], Loss: 0.7063536643981934, Training acc: 0.5480182926829268
Epoch[2], Batch[5

## Output Result

In [15]:
class OutputDataset(Dataset):
  def __init__(self, image_dir, transform):
    super().__init__()
    self.image_dir = image_dir
    self.transform = transform
    self.images = os.listdir(self.image_dir)

  def __len__(self):
    return len(self.images)

  def __getitem__(self, i):
    image_path = os.path.join(self.image_dir, self.images[i])
    image = Image.open(image_path).convert('L')
    image = self.transform(image)
    label = int(self.images[i][1])
    stock = self.images[i].split('_')[1]
    date = self.images[i].split('_')[2].replace('.png', '')
    return (image, label, stock, date)

In [16]:
training_data = OutputDataset('img_span12/training', T.Compose([T.Resize((224, 224)), T.ToTensor()]))
testing_data = OutputDataset('img_span12/testing', T.Compose([T.Resize((224, 224)), T.ToTensor()]))

In [17]:
train_loader = DataLoader(training_data, batch_size=32, shuffle=False)
test_loader = DataLoader(testing_data, batch_size=32, shuffle=False)

In [18]:
def extract_vectors(model, dataloader):
  model.eval()
  all_records = []
  total_correct = 0
  total_samples = 0

  with torch.no_grad():
    for x, y, stocks, dates in dataloader:
      x, y = x.to(device), y.to(device)
      outputs, scores = model(x)
      outputs = outputs.cpu().numpy()

      predictions = scores.max(1)[1]
      total_correct += predictions.eq(y).sum().item()
      total_samples += y.size(0)

      for i in range(len(dates)):
        record = {
            'stock': stocks[i],
            'date': dates[i],
            'vector': ','.join(map(str, outputs[i])),
            'label': y[i].item()
        }
        all_records.append(record)
    print('Acc:', total_correct/total_samples)
  return all_records

In [19]:
# training datasets
records = extract_vectors(model, train_loader)

df = pd.DataFrame(records)
df.sort_values(by=['stock', 'date'])
print(df)

# Save the DataFrame to a CSV file
df.to_csv('output/ResNet_output_vectors_training.csv', index=True)

Acc: 0.8869093725793958
     stock        date                                             vector  \
0     2912  2017-02-10  -0.4077851,0.19726984,-0.74746937,0.26134998,0...   
1     2912  2017-03-02  -0.65246403,0.35797524,-1.7846167,1.079017,1.0...   
2     2912  2017-03-20  -0.5088743,0.459578,-2.4450994,0.87230116,1.71...   
3     2912  2017-04-07  -1.1687809,0.06394574,-1.4299024,0.47921085,0....   
4     2912  2017-04-25  -0.73606294,0.44391906,-1.3083328,0.5477185,0....   
...    ...         ...                                                ...   
7741  1303  2018-05-24  -0.9574906,-0.24473104,-0.70676005,0.14488757,...   
7742  1303  2018-06-11  -0.44444862,0.22950763,-0.8920603,0.30187696,0...   
7743  1303  2018-06-28  -0.51732576,0.41528252,-1.0812718,0.57271475,0...   
7744  1303  2018-07-16  -0.5512857,0.4888211,-1.4384528,0.73857975,0.8...   
7745  1303  2018-08-01  -0.9803419,0.23620172,-1.6258641,0.46136278,0....   

      label  
0         0  
1         1  
2        

In [20]:
# testing datasets
records = extract_vectors(model, test_loader)

df = pd.DataFrame(records)
df.sort_values(by=['stock', 'date'])
print(df)

# Save the DataFrame to a CSV file
df.to_csv('output/ResNet_output_vectors_testing.csv', index=True)

Acc: 0.5114893617021277
     stock        date                                             vector  \
0     2615  2022-01-10  -0.793993,0.045899004,-0.5591482,0.103506684,0...   
1     2615  2022-01-26  -0.7627259,-0.09119034,-0.41103378,-0.21178478...   
2     2615  2022-02-22  -0.282455,0.1629546,-0.8225675,0.3258345,0.517...   
3     2615  2022-03-11  -0.35823098,0.18023129,-0.4616228,0.191158,0.3...   
4     2615  2022-03-29  -0.66777825,0.15947549,-0.7235345,0.2166799,0....   
...    ...         ...                                                ...   
2345  1402  2024-04-09  -0.6877149,0.23608264,-0.8583309,0.4060331,0.4...   
2346  1402  2024-04-25  -0.9802896,0.12793365,-0.76728565,0.11412499,0...   
2347  1402  2024-05-14  -0.4336423,0.2250413,-0.86563003,0.37150875,0....   
2348  1402  2024-05-30  -1.032095,0.5834658,-2.136323,0.8411648,0.7425...   
2349  1402  2024-06-18  -0.41709688,0.12631299,-0.51364434,0.35680038,...   

      label  
0         0  
1         1  
2        